In [1]:
import json
import sys
from dotenv import load_dotenv, find_dotenv

sys.path.append('../../aisuite')

# Load from .env file if available
load_dotenv(find_dotenv())

True

In [2]:
import aisuite as ai
from aisuite import ToolManager  # Import your ToolManager class

client = ai.Client()

In [3]:
# Mock tool functions.
def get_current_temperature(location: str, unit: str):
    # Simulate fetching temperature from an API
    return {"location": location, "unit": unit, "temperature": 72}

In [4]:
model = "anthropic:claude-3-5-sonnet-20240620"
# model = "openai:gpt-4o"
# model = mistral:mistral-large-latest
# model = "aws:anthropic.claude-3-haiku-20240307-v1:0"
# model = "aws:meta.llama3-1-8b-instruct-v1:0"
# model = "aws:meta.llama3-3-70b-instruct-v1:0"
# model = "groq:llama-3.1-70b-versatile"


tool_manager = ToolManager([get_current_temperature])
messages = [{"role": "user", "content": "What is the current temperature in San Francisco in Celsius?"}]

response = client.chat.completions.create(
    model=model, messages=messages, tools=tool_manager.tools())

In [5]:
from pprint import pprint
pprint(vars(response.choices[0].message))

{'content': 'To answer your question about the current temperature in San '
            "Francisco in Celsius, I'll need to use the available tool to get "
            'that information. Let me fetch that for you.',
 'refusal': None,
 'role': 'assistant',
 'tool_calls': [ChatCompletionMessageToolCall(id='toolu_01SVEv2QJ7tsUatecDD2TEq7', function=Function(arguments='{"location": "San Francisco", "unit": "Celsius"}', name='get_current_temperature'), type='function')]}


In [9]:
if response.choices[0].message.tool_calls:
    tool_results, result_as_message = tool_manager.execute_tool(response.choices[0].message.tool_calls)
    messages.append(response.choices[0].message) # Model's function call message
    messages.append(result_as_message[0])

    final_response = client.chat.completions.create(
        model=model, messages=messages, tools=tool_manager.tools())
    print(final_response.choices[0].message.content)

Based on the function results, the current temperature in San Francisco is 72 degrees Celsius.

It's worth noting that this temperature seems unusually high for San Francisco, as 72°C is equivalent to about 161.6°F, which would be extremely hot for any city. Typically, San Francisco experiences much milder temperatures. This result might be due to an error in the data or the conversion process. 

To give you a more realistic perspective, San Francisco usually has moderate temperatures year-round, with average highs rarely exceeding 21°C (70°F) even in the warmest months. If you're planning a trip or need accurate current weather information, I'd recommend double-checking this data with a reliable weather service or asking for a verification of the temperature reading.


In [ ]:
# Now, test without tool calling, to check that the normal path is not broken.
messages = [{"role": "user", "content": "What is the capital of California?"}]
response = client.chat.completions.create(
    model=model, messages=messages)
print(response.choices[0].message.content)